In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
import os
import matplotlib.pyplot as plt
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgbm

In [3]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, GaussianNoise
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


# 데이터 불러오기

In [4]:
# 데이터 불러오기
train = pd.read_csv('/home/lab21/data/train.csv', index_col=0)
train_2 = pd.read_csv('/home/lab21/data/train_x_0.2_99.8.csv', index_col=0)

test = pd.read_csv('/home/lab21/data/test.csv', index_col=0)
sample_submission = pd.read_csv('/home/lab21/data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
#x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = train_2.drop(columns=['fiberID'], axis=1)

test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [5]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [6]:
x=x.astype('float32')
test_x=test_x.astype('float32')

# Train_Test_SpIit

In [7]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, \
                                                    train_size=0.8, shuffle=True ,random_state=42)

# Scaler

In [8]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

# Variables

In [9]:
import datetime
start=datetime.datetime.now()
print(start)

2020-02-25 01:25:20.083066


In [ ]:
training_accuracy = []
test_accuracy = []
# 1에서 10까지 n_neighbors를 적용

neighbors_settings = range(10, 55, 1)
#neighbors_settings = [ 2, 6, 10]

for n_neighbors in neighbors_settings:
    print(n_neighbors)
    # 모델 생성
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs=24)
    clf.fit(x_train, y_train)
    # 훈련 세트 정확도 저장
    training_accuracy.append(clf.score(x_train, y_train))
    # 일반화 정확도 저장
    test_accuracy.append(clf.score(x_test, y_test))

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


In [ ]:
plt.plot(neighbors_settings, training_accuracy, label="train acc")
plt.plot(neighbors_settings, test_accuracy, label="test acc")
plt.ylabel("accuracy")
plt.xlabel("n_neighbors")
plt.legend()

In [ ]:
for cnt,i in enumerate(neighbors_settings):
    print(i,training_accuracy[cnt],test_accuracy[cnt])

In [ ]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)


In [ ]:
clf = KNeighborsClassifier(n_neighbors=3, n_jobs=24)
clf.fit(x_train, y_train)
pred=clf.predict(test_x)
xxx=clf.predict_proba(test_x)
for i in range(1000):
    print(pred[i])
    #print(i,to_categorical(pred[i],19))
    #print(i,xxx[i],'\n')

# Fianl K_Number: 14

# Model Predict

In [ ]:
x=pd.read_csv('/home/lab21/20200219/csv/submission_data_KerasClassifier_1.csv',index_col=False)

In [ ]:
del x['id']

In [ ]:
y=x.copy()

In [ ]:
y=np.array(y)

In [ ]:

ac=[]
for k in range(2,55):
    acc=0
    clf = KNeighborsClassifier(n_neighbors=k, n_jobs=24)
    clf.fit(x_train, y_train)
    pred=clf.predict(test_x)
    for i in range(10009):

        tmp1=np.argmax(y[i])
        tmp2=pred[i]
        if tmp1 == tmp2 :
            acc+=1
    ac.append(acc/10009.)
    print(k,acc/10009.)
print('end')



In [ ]:
ac